In [4]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter

load_dotenv()
api_key  = os.getenv("OPENAI_API_KEY")

# current_dir = os.path.dirname( os.getcwd() )
# print("Dir:", current_dir)

# Loading knowledge retriever
loader = TextLoader("retrieval/returning.txt", encoding="UTF-8")
data = loader.load()
# text_splitter = CharacterTextSplitter(
#     separator="\n\n",
#     chunk_size=300,
#     length_function=len
# )

text_splitter = RecursiveCharacterTextSplitter(
chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
)
docs = text_splitter.split_documents(data)
print("Chunks: ", len(docs))
embedings_model = OpenAIEmbeddings(model = "text-embedding-3-small")

db = FAISS.from_documents(docs,embedings_model)
retriever = db.as_retriever(search_kwargs={"k": 1})

from langchain.agents.agent_toolkits import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "emag_return_policies",
    "Searches and returns information about return questions for Emag.",
)
tools = [retriever_tool]


llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0,streaming=True)


# Define prompt
from langchain.prompts import MessagesPlaceholder
from langchain.schema.messages import SystemMessage

system_message = SystemMessage(
    content=(
        """You are a customer support bot for a bulgarian e-commerce store called EMAG ."
            You give help to questions regarding the return of items to emag.
            Answer the questions only based on your knowledge and do not in any case make up information.
            Answer the questions briefly and summarize your information.
            Say maximum 3 sentances per answer
            Аnswer the questions in bulgarian.
            Only answer based on your context.
            Answer briefly!"""
    )
)


from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
memory_key = "chat_history"
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)],
)

# Define final Agent 
from langchain.agents import create_openai_functions_agent

#agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
agent = create_openai_functions_agent(llm, tools, prompt)

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    return_intermediate_steps=False,
    verbose=True
)


from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
message_history = ChatMessageHistory()
message_history.clear()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key=memory_key,
)



Chunks:  18


In [5]:
agent_with_chat_history.invoke({'input': 'Здравейте как да върна продукт?'}, {'configurable': {'session_id': 'asd'}})['output']



> Entering new AgentExecutor chain...

Invoking: `emag_return_policies` with `{'query': 'връщане на продукт'}`


продукта в първоначалното му състояние от гледна точка на софтуера;
       Не можеш да върнеш продукта, ако е с активиран OEM лиценз, или ако оперативните системи или други програми, с които е бил доставен продукта са деинсталирани;
       Можеш да върнеш защитните фолиа за таблети и телефони само ако не са използвани;
Принтери, скенери, консумативи
        Не могат да бъдат върнати принтери или мултифункционални устройства, чиито консумативи са били разпечатани;
        Могат да бъдат върнати консумативи като батерии, касети за принтер, аксесоари и офис консумативи, само ако са запечатани;
Продукти от категорията Мода
 Ако желаеш да върнеш продукт от тази категория, опиши ситуацията в полето за забележки и избери желаната от теб опция във Формуляра за връщане;

Избери една от опциите за връщане: възстановяване на заплатената сума или ваучер, който може да бъде използван з

'За да върнете продукт, трябва да го върнете в първоначалното му състояние и да не е активиран OEM лиценз. Защитните фолиа за таблети и телефони могат да бъдат върнати само ако не са използвани. Принтери и консумативи могат да бъдат върнати само ако са в запечатано състояние.'